# Sampling NLCD

This code was written with the assistance of Google Gemini's Flash 2.5. Assistance was used between Monday June 2, 2025 and Friday June 6, 2025 to help with the implementation of the sampling algorithm. The method of sampling was constructed to mimic the sampling procedure of the actual NRI survey and was the idea of the author, Caleb Leedy.

In [58]:
import rasterio
import numpy as np
import pandas as pd
import random
from rasterio.transform import xy
import pyproj

def process_tiff_image(file_path):
    """
    Processes a single-band TIFF image according to the specified steps:
    1. Divides into non-overlapping 360x360 squares.
    2. Removes squares with > 5% zero pixels.
    3. Selects six 60x60 sections from unique rows/columns within remaining squares.
    4. Randomly selects a 15x15 quarter section from each 60x60 section.
    5. Selects three specific pixels from each 15x15 quarter section.
    6. Outputs a DataFrame with information about the final selected pixels.

    Args:
        file_path (str): Path to the input single-band TIFF file.
        output_dataframe_path (str): Path to save the resulting DataFrame as a CSV.

    Returns:
        pandas.DataFrame: DataFrame containing information about the selected pixels.
                          Returns an empty DataFrame on error or if no pixels are selected.
    """
    # 1. Load the TIFF image
    try:
        with rasterio.open(file_path) as src:
            if src.count != 1:
                print(f"Warning: TIFF has {src.count} bands. Processing only the first band.")
            original_image_data = src.read(1)
            original_rows, original_cols = original_image_data.shape
            print(f"Loaded image: {original_rows} rows, {original_cols} columns.")
    except rasterio.errors.RasterioIOError as e:
        print(f"Error loading TIFF file: {e}. Please check path and file integrity.")
        return pd.DataFrame()
    except Exception as e:
        print(f"An unexpected error occurred while loading image: {e}")
        return pd.DataFrame()

    # Define sizes
    square_360_size = 360
    square_60_size = 60
    square_15_size = 15

    # Lists to store intermediate and final results
    all_360_squares = [] # Stores (360_square_data, original_row_start, original_col_start, square_360_id)
    
    # 2. Divide the original image into non-overlapping squares with side lengths of 360 pixels.
    square_360_id_counter = 0
    num_rows_360 = original_rows // square_360_size
    num_cols_360 = original_cols // square_360_size

    print(f"Dividing image into {num_rows_360}x{num_cols_360} 360x360 squares...")

    for r_idx_360 in range(num_rows_360):
        for c_idx_360 in range(num_cols_360):
            row_start_orig = r_idx_360 * square_360_size
            col_start_orig = c_idx_360 * square_360_size
            
            square_360_data = original_image_data[
                row_start_orig : row_start_orig + square_360_size,
                col_start_orig : col_start_orig + square_360_size
            ]
            all_360_squares.append((square_360_data, row_start_orig, col_start_orig, square_360_id_counter))
            square_360_id_counter += 1
    print(f"Extracted {len(all_360_squares)} 360x360 squares.")

    # 3. Remove squares with more than 5% of the pixels equal to "0".
    filtered_360_squares = []
    zero_threshold_percentage = 5
    total_pixels_360 = square_360_size * square_360_size
    max_allowed_zeros = (zero_threshold_percentage / 100.0) * total_pixels_360

    print(f"Filtering 360x360 squares (max {zero_threshold_percentage}% zeros allowed)...")

    for square_data, r_orig, c_orig, s360_id in all_360_squares:
        num_zeros = np.sum(square_data == 0)
        if num_zeros <= max_allowed_zeros:
            filtered_360_squares.append((square_data, r_orig, c_orig, s360_id))
    print(f"Remaining 360x360 squares after filtering: {len(filtered_360_squares)}")

    # Prepare for DataFrame creation
    selected_pixel_data = [] # List of dictionaries for DataFrame rows

    # 4. Process the remaining 360x360 squares
    for square_360_data, orig_row_360_start, orig_col_360_start, s360_id in filtered_360_squares:
        
        # Determine the number of 60x60 sections within a 360x360 square
        num_60_sections_per_side = square_360_size // square_60_size # Should be 6
        
        # 4. In the remaining squares select six sections of 60 by 60 pixels
        #    such that each selected section is from a unique column and row.
        # Simplest way to ensure unique row/column is to pick sections along the diagonal.
        # e.g., (0,0), (1,1), (2,2), (3,3), (4,4), (5,5) in the 6x6 grid of sections
        # FIXME: This is incorrect. I want a random selection of unique rows and columns
        
        selected_60_sections_info = [] # Stores (60_section_data, relative_row_start, relative_col_start, s60_id)
        section_row_ind = np.arange(num_60_sections_per_side)
        section_col_ind = np.arange(num_60_sections_per_side)
        random.shuffle(section_col_ind)

        for i, j in zip(section_row_ind, section_col_ind):
            # Calculate top-left corner relative to the 360x360 square
            relative_row_60_start = i * square_60_size
            relative_col_60_start = j * square_60_size
            
            section_60_data = square_360_data[
                relative_row_60_start : relative_row_60_start + square_60_size,
                relative_col_60_start : relative_col_60_start + square_60_size
            ]
            # s60_id can be derived from its (row_idx, col_idx) in the 6x6 grid
            s60_id = f"({i},{j})" 
            selected_60_sections_info.append((section_60_data, relative_row_60_start, relative_col_60_start, s60_id))

        # 4. Randomly select a quarter section of 15 by 15 pixels from each selected 60x60 section.
        for s60_data, rel_row_60_start, rel_col_60_start, s60_id in selected_60_sections_info:
            
            # Possible top-left offsets for 15x15 quarters within a 60x60 section
            # (row_offset, col_offset) relative to 60x60 section
            quarter_offsets = [
                (0, 0), # Top-Left
                (0, square_15_size), # Top-Right (col 15)
                (square_15_size, 0), # Bottom-Left (row 15)
                (square_15_size, square_15_size) # Bottom-Right (row 15, col 15)
            ]
            
            # Randomly choose one quarter
            random_q_offset_row, random_q_offset_col = random.choice(quarter_offsets)
            
            # Extract the 15x15 quarter section
            square_15_data = s60_data[
                random_q_offset_row : random_q_offset_row + square_15_size,
                random_q_offset_col : random_q_offset_col + square_15_size
            ]
            # s15_id indicates which quarter was chosen (e.g., "TL", "TR", "BL", "BR")
            s15_id_map = {(0,0):"TL", (0,15):"TR", (15,0):"BL", (15,15):"BR"}
            s15_id = s15_id_map.get((random_q_offset_row, random_q_offset_col), "Unknown")

            # 5. In each selected quarter section, select three pixels
            #    such that one pixel from the top third, one pixel from the middle third
            #    and one pixel is from the bottom third while each selected pixel is
            #    contained within its own column third (left, middle, or center).

            # Define row and column thirds (relative to 15x15 square)
            row_thirds_ranges = [(0, 4), (5, 9), (10, 14)] # (start_row, end_row) inclusive
            col_thirds_ranges = [(0, 4), (5, 9), (10, 14)] # (start_col, end_col) inclusive

            # To ensure unique row and column thirds for the 3 selected pixels,
            # we fix the row thirds (top, middle, bottom) and randomly permute
            # the column thirds (left, middle, right).
            col_third_indices_permutation = list(range(3)) # [0, 1, 2]
            random.shuffle(col_third_indices_permutation) # e.g., [2, 0, 1]

            for i in range(3): # For each of the three pixels
                row_third_idx = i # 0 (top), 1 (middle), 2 (bottom)
                col_third_idx = col_third_indices_permutation[i] # Randomly assigned column third

                row_min, row_max = row_thirds_ranges[row_third_idx]
                col_min, col_max = col_thirds_ranges[col_third_idx]

                # Randomly select a row and column within the specific third
                selected_row_in_15 = random.randint(row_min, row_max)
                selected_col_in_15 = random.randint(col_min, col_max)

                # Calculate the original image row and column index
                original_row_index = (
                    orig_row_360_start +      # Start of 360 square
                    rel_row_60_start +        # Start of 60 square within 360
                    random_q_offset_row +     # Start of 15 square within 60
                    selected_row_in_15        # Relative row in 15 square
                )
                original_col_index = (
                    orig_col_360_start +      # Start of 360 square
                    rel_col_60_start +        # Start of 60 square within 360
                    random_q_offset_col +     # Start of 15 square within 60
                    selected_col_in_15        # Relative col in 15 square
                )

                selected_pixel_data.append({
                    'township': s360_id,
                    'section': s60_id,
                    'quartersection': s15_id,
                    'nlcd_value': square_15_data[selected_row_in_15, selected_col_in_15], # Value of the selected pixel
                    'original_row_index': original_row_index,
                    'original_col_index': original_col_index
                })

    # 5. The output should be a data frame
    df = pd.DataFrame(selected_pixel_data)
    print(f"\nTotal selected pixels for DataFrame: {len(df)}")
    
    return df

In [59]:
random.seed(123)
np.random.seed(123)
df = process_tiff_image("data/raw/NLCD_MA_2016.tif")

Loaded image: 6169 rows, 10004 columns.
Dividing image into 17x27 360x360 squares...
Extracted 459 360x360 squares.
Filtering 360x360 squares (max 5% zeros allowed)...
Remaining 360x360 squares after filtering: 142

Total selected pixels for DataFrame: 2556


In [60]:
df

,township,section,quartersection,nlcd_value,original_row_index,original_col_index
0,45,"(0,3)",TL,24,364,6674
1,45,"(0,3)",TL,41,367,6667
2,45,"(0,3)",TL,41,370,6661
3,45,"(1,4)",TR,23,421,6736
4,45,"(1,4)",TR,23,425,6748
...,...,...,...,...,...,...
2551,370,"(4,3)",TR,90,4925,7048
2552,370,"(4,3)",TR,90,4933,7035
2553,370,"(5,5)",BR,41,4996,7163
2554,370,"(5,5)",BR,95,5003,7157


In [61]:
def get_lat_lon_from_pixel(pixel_row, pixel_col, src_transform, src_crs, transformer=None):
    """
    Converts a single pixel's (row, col) to its latitude and longitude.
    
    Args:
        pixel_row (float): The row index of the pixel center (e.g., 0.5 for top-left).
        pixel_col (float): The column index of the pixel center (e.g., 0.5 for top-left).
        src_transform (affine.Affine): The affine transformation matrix from rasterio.
        src_crs (rasterio.crs.CRS): The CRS of the source raster.
        transformer (pyproj.Transformer, optional): A pre-initialized pyproj Transformer
                                                     for efficiency if many conversions are needed.
    Returns:
        tuple: (latitude, longitude) of the pixel center.
    """
    # Convert pixel (row, col) to world (x, y) coordinates
    x_world, y_world = xy(src_transform, pixel_row, pixel_col)

    latitude, longitude = None, None

    if src_crs.is_geographic:
        longitude = x_world
        latitude = y_world
    else:
        # If no transformer is provided, create one
        if transformer is None:
            transformer = pyproj.Transformer.from_crs(
                crs_from=src_crs,
                crs_to="EPSG:4326", # WGS84 Lat/Lon
                always_xy=True # Ensures output is (longitude, latitude)
            )
        longitude, latitude = transformer.transform(x_world, y_world)
    
    return latitude, longitude


In [62]:
file_path = "data/raw/NLCD_MA_2016.tif"
with rasterio.open(file_path) as src:
    transform = src.transform # Get the affine transform
    crs = src.crs # Get the Coordinate Reference System

df[['latitude', 'longitude']] = df.apply(lambda row: get_lat_lon_from_pixel(row["original_row_index"], row["original_col_index"], transform, crs), axis=1, result_type='expand')

In [63]:
df.head(20)

,township,section,quartersection,nlcd_value,original_row_index,original_col_index,latitude,longitude
0,45,"(0,3)",TL,24,364,6674,42.805040,-71.110206
1,45,"(0,3)",TL,41,367,6667,42.804239,-71.112778
2,45,"(0,3)",TL,41,370,6661,42.803436,-71.114984
3,45,"(1,4)",TR,23,421,6736,42.789569,-71.087569
4,45,"(1,4)",TR,23,425,6748,42.788473,-71.083176
5,45,"(1,4)",TR,22,430,6744,42.787128,-71.084652
6,45,"(2,1)",BR,21,497,6563,42.769254,-71.151114
7,45,"(2,1)",BR,90,500,6565,42.768441,-71.150386
8,45,"(2,1)",BR,41,505,6556,42.767101,-71.153692
9,45,"(3,5)",TR,22,543,6809,42.756524,-71.061034


In [65]:
df.to_csv("data/clean/NLCD_MA_2016_sample.csv", index=False)